In [153]:
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite

from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma as kkm
from konlpy.tag import Okt
from konlpy.tag import Mecab

In [55]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [188]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [189]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1=dict(ind_reviews(1001))

In [190]:
reviews1

{'태백에 있다는 위치적인 단점은 있지만 일을 배우기에 좋은 병원': '70',
 '뚜렷한 목표의식이 있다면 좋은 경험과 경력이 될 수 있는 곳!': '70',
 '월급 중요하지 않고 오로지 정년 보장만 중요하다면 공무원 추천함': '70',
 '일단 파벌싸움이심하다 진짜심각하다 매우심각하다 \r그러나 직원복지는 작살난다': '70',
 '주먹구구식에다 인수인계 개판임, 워라밸 보장도 안됨': '70',
 '지방 공공기관이지만 수직적이지 않음. 신입도 일하기 편함. 워라벨 좋음.': '70',
 '서민금융생활법에 따라 9월 특별 법인화가 확정된 채무조정기구- 채무 감면 및 소액금융등이 주 업무': '70',
 '기술뿐만이 아니라 정책적인 부분까지 모두 도맡아 할 수 있는 곳': '70',
 '- 고시/비고시 출신 가림- 행정직이 일하기 좋음 그 외 직렬은 X- 업무량 많음- 조직이 커서 여러 업무 배우기 좋음': '70',
 '재외공관 대표부에서 계약직으로 근무. 태극마크라는 자부심으로 일하는 조직': '70',
 '서울시립 청소년수련시설로 청소년활동 프로그램운영 및 지원, 평생교육프로그램 운영, 천문과학특화 프로그램 운영 등을 주요 사업으로 하고 있으며, 직원간의 유대 및 협력관계가 좋음': '38',
 '기관장 말에 이리저리 휘둘리고 싶으면 입사하세요 기관장 말이 곧 법인 곳입니다(올해 리뷰들 조작아닌가 의심될 정도로 황당..)': '38',
 '울산문화재단 CCTV 불법사찰, 감시에 대해 정부는 특검을 실시하라! 건전한 조직문화 해치는 불법행위 근절하라!! <cctv 불법사찰 규탄 운동>': '38',
 '체계가 없는 회사이고 나름대로 분위기는 수평적인것같습니다': '38',
 '끊임없는 삽질과 고성 그리고 독재와 이기주의가 팽배한 행복한 우리 회사': '38',
 '안정적인 공공기관. 퇴사한지 조금 되었는데 그때는 괜찮았습니다.': '38',
 '근본없는 회사라 어떤일이 일어나도 납득이 됨.조례도 개판이고 뭐 제대로 갖춰진게 하나도 없음.': '38',

In [191]:
text1=list(reviews1.keys())
score1=list(reviews1.values())

In [221]:
import torch

In [222]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

ModuleNotFoundError: No module named 'kobert'

In [ ]:
input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])

In [223]:
import onnxruntime

In [224]:
import numpy as np

In [226]:
from kobert.utils import get_onnx

ModuleNotFoundError: No module named 'kobert'

In [258]:
n=1
sent_number="Sentence: {}".format(n)
print(sent_number)

Sentence: 1


In [213]:
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  64.133


In [273]:
sent_num=0
text1_sent=[]
for i in range(len(text1)):
    text1_pos=okt.pos(text1[i])
    sent_num+=1
    for t in text1_pos:
        n=list(t)
        sent_number="Sentence: {}".format(sent_num)
        n.insert(0, sent_number)
        n.insert(3, score1[(sent_num-1)]) # 점수
        text1_sent.append(n)

pprint.pprint(text1_sent)

[['Sentence: 1', '태백', 'Noun', '70'],
 ['Sentence: 1', '에', 'Josa', '70'],
 ['Sentence: 1', '있다는', 'Adjective', '70'],
 ['Sentence: 1', '위치', 'Noun', '70'],
 ['Sentence: 1', '적', 'Suffix', '70'],
 ['Sentence: 1', '인', 'Josa', '70'],
 ['Sentence: 1', '단점', 'Noun', '70'],
 ['Sentence: 1', '은', 'Josa', '70'],
 ['Sentence: 1', '있지만', 'Adjective', '70'],
 ['Sentence: 1', '일', 'Noun', '70'],
 ['Sentence: 1', '을', 'Josa', '70'],
 ['Sentence: 1', '배우기에', 'Verb', '70'],
 ['Sentence: 1', '좋은', 'Adjective', '70'],
 ['Sentence: 1', '병원', 'Noun', '70'],
 ['Sentence: 2', '뚜렷한', 'Adjective', '70'],
 ['Sentence: 2', '목표', 'Noun', '70'],
 ['Sentence: 2', '의식', 'Noun', '70'],
 ['Sentence: 2', '이', 'Josa', '70'],
 ['Sentence: 2', '있다면', 'Adjective', '70'],
 ['Sentence: 2', '좋은', 'Adjective', '70'],
 ['Sentence: 2', '경험', 'Noun', '70'],
 ['Sentence: 2', '과', 'Josa', '70'],
 ['Sentence: 2', '경력', 'Noun', '70'],
 ['Sentence: 2', '이', 'Josa', '70'],
 ['Sentence: 2', '될', 'Verb', '70'],
 ['Sentence: 2', '수', 

 ['Sentence: 62', '못', 'VerbPrefix', '76'],
 ['Sentence: 62', '하는', 'Verb', '76'],
 ['Sentence: 62', '곳', 'Noun', '76'],
 ['Sentence: 62', '.', 'Punctuation', '76'],
 ['Sentence: 63', '안정', 'Noun', '76'],
 ['Sentence: 63', '적', 'Suffix', '76'],
 ['Sentence: 63', '인', 'Josa', '76'],
 ['Sentence: 63', '직장', 'Noun', '76'],
 ['Sentence: 63', '이나', 'Josa', '76'],
 ['Sentence: 63', '그만큼', 'Adverb', '76'],
 ['Sentence: 63', '폐쇄적', 'Noun', '76'],
 ['Sentence: 63', '인', 'Josa', '76'],
 ['Sentence: 63', '조직', 'Noun', '76'],
 ['Sentence: 64', '최소한', 'Noun', '76'],
 ['Sentence: 64', '의', 'Josa', '76'],
 ['Sentence: 64', '사회', 'Noun', '76'],
 ['Sentence: 64', '적', 'Suffix', '76'],
 ['Sentence: 64', '관계', 'Noun', '76'],
 ['Sentence: 64', '에서', 'Josa', '76'],
 ['Sentence: 64', '본인', 'Noun', '76'],
 ['Sentence: 64', '업무', 'Noun', '76'],
 ['Sentence: 64', '를', 'Josa', '76'],
 ['Sentence: 64', '할', 'Verb', '76'],
 ['Sentence: 64', '수', 'Noun', '76'],
 ['Sentence: 64', '있는', 'Adjective', '76'],
 ['Senten

 ['Sentence: 109', '해야하는', 'Verb', '36'],
 ['Sentence: 109', '일이', 'Modifier', '36'],
 ['Sentence: 109', '너무', 'Adverb', '36'],
 ['Sentence: 109', '많은', 'Adjective', '36'],
 ['Sentence: 109', '곳', 'Noun', '36'],
 ['Sentence: 109', '대신', 'Noun', '36'],
 ['Sentence: 109', '여러', 'Modifier', '36'],
 ['Sentence: 109', '업무', 'Noun', '36'],
 ['Sentence: 109', '를', 'Josa', '36'],
 ['Sentence: 109', '하면서', 'Verb', '36'],
 ['Sentence: 109', '다양한', 'Adjective', '36'],
 ['Sentence: 109', '경험', 'Noun', '36'],
 ['Sentence: 109', '가능', 'Noun', '36'],
 ['Sentence: 110', '과로', 'Noun', '36'],
 ['Sentence: 110', '가', 'Josa', '36'],
 ['Sentence: 110', '일상', 'Noun', '36'],
 ['Sentence: 110', '이며', 'Josa', '36'],
 ['Sentence: 110', ',', 'Punctuation', '36'],
 ['Sentence: 110', '군대', 'Noun', '36'],
 ['Sentence: 110', '식', 'Suffix', '36'],
 ['Sentence: 110', '폐쇄적', 'Noun', '36'],
 ['Sentence: 110', '문화', 'Noun', '36'],
 ['Sentence: 110', '가', 'Josa', '36'],
 ['Sentence: 110', '강한', 'Adjective', '36'],
 ['Sent

 ['Sentence: 142', '은', 'Josa', '46'],
 ['Sentence: 142', '안되는', 'Adjective', '46'],
 ['Sentence: 142', '...', 'Punctuation', '46'],
 ['Sentence: 143', '대학생', 'Noun', '46'],
 ['Sentence: 143', '인턴', 'Noun', '46'],
 ['Sentence: 143', '용', 'Noun', '46'],
 ['Sentence: 143', '으로는', 'Josa', '46'],
 ['Sentence: 143', '경험', 'Noun', '46'],
 ['Sentence: 143', '하기', 'Verb', '46'],
 ['Sentence: 143', '나쁘지', 'Adjective', '46'],
 ['Sentence: 143', '않지만', 'Verb', '46'],
 ['Sentence: 143', ',', 'Punctuation', '46'],
 ['Sentence: 143', '정규직', 'Noun', '46'],
 ['Sentence: 143', '으로', 'Josa', '46'],
 ['Sentence: 143', '들어가는건', 'Verb', '46'],
 ['Sentence: 143', '한번', 'Noun', '46'],
 ['Sentence: 143', '쯤', 'Suffix', '46'],
 ['Sentence: 143', '고민', 'Noun', '46'],
 ['Sentence: 143', '해봐', 'Verb', '46'],
 ['Sentence: 143', '야할듯', 'Adjective', '46'],
 ['Sentence: 144', '계약', 'Noun', '46'],
 ['Sentence: 144', '직', 'Noun', '46'],
 ['Sentence: 144', '의', 'Josa', '46'],
 ['Sentence: 144', '비애', 'Noun', '46'],
 ['S

 ['Sentence: 199', '\r', 'Foreign', '58'],
 ['Sentence: 199', '(', 'Punctuation', '58'],
 ['Sentence: 199', '인턴', 'Noun', '58'],
 ['Sentence: 199', '이라', 'Josa', '58'],
 ['Sentence: 199', '권한', 'Noun', '58'],
 ['Sentence: 199', '이', 'Josa', '58'],
 ['Sentence: 199', '없어', 'Adjective', '58'],
 ['Sentence: 199', '이렇게', 'Adverb', '58'],
 ['Sentence: 199', '느낀', 'Verb', '58'],
 ['Sentence: 199', '걸수도', 'Verb', '58'],
 ['Sentence: 199', ')', 'Punctuation', '58'],
 ['Sentence: 200', '울산', 'Noun', '58'],
 ['Sentence: 200', '광역시', 'Noun', '58'],
 ['Sentence: 200', ',', 'Punctuation', '58'],
 ['Sentence: 200', '군', 'Noun', '58'],
 ['Sentence: 200', ',', 'Punctuation', '58'],
 ['Sentence: 200', '구', 'Noun', '58'],
 ['Sentence: 200', '에서', 'Josa', '58'],
 ['Sentence: 200', '필요한', 'Adjective', '58'],
 ['Sentence: 200', '내용', 'Noun', '58'],
 ['Sentence: 200', '에', 'Josa', '58'],
 ['Sentence: 200', '대한', 'Noun', '58'],
 ['Sentence: 200', '연구', 'Noun', '58'],
 ['Sentence: 200', '와', 'Josa', '58'],
 [

 ['Sentence: 251', '있어선', 'Adjective', '56'],
 ['Sentence: 251', '편해요', 'Adjective', '56'],
 ['Sentence: 252', '사내정치', 'Noun', '56'],
 ['Sentence: 252', '질', 'Noun', '56'],
 ['Sentence: 252', '이', 'Josa', '56'],
 ['Sentence: 252', '아주', 'Noun', '56'],
 ['Sentence: 252', '중요한', 'Adjective', '56'],
 ['Sentence: 252', '곳', 'Noun', '56'],
 ['Sentence: 252', '휩쓸', 'Adjective', '56'],
 ['Sentence: 252', '리', 'Noun', '56'],
 ['Sentence: 252', '면', 'Josa', '56'],
 ['Sentence: 252', '끝', 'Noun', '56'],
 ['Sentence: 253', '계약', 'Noun', '56'],
 ['Sentence: 253', '직', 'Noun', '56'],
 ['Sentence: 253', '과', 'Josa', '56'],
 ['Sentence: 253', '정규직', 'Noun', '56'],
 ['Sentence: 253', '이', 'Josa', '56'],
 ['Sentence: 253', '너무', 'Adverb', '56'],
 ['Sentence: 253', '확연', 'Noun', '56'],
 ['Sentence: 253', '한', 'Josa', '56'],
 ['Sentence: 253', '곳', 'Noun', '56'],
 ['Sentence: 253', ',', 'Punctuation', '56'],
 ['Sentence: 253', '성골', 'Noun', '56'],
 ['Sentence: 253', '과', 'Josa', '56'],
 ['Sentence: 253',

In [274]:
text1_sent

[['Sentence: 1', '태백', 'Noun', '70'],
 ['Sentence: 1', '에', 'Josa', '70'],
 ['Sentence: 1', '있다는', 'Adjective', '70'],
 ['Sentence: 1', '위치', 'Noun', '70'],
 ['Sentence: 1', '적', 'Suffix', '70'],
 ['Sentence: 1', '인', 'Josa', '70'],
 ['Sentence: 1', '단점', 'Noun', '70'],
 ['Sentence: 1', '은', 'Josa', '70'],
 ['Sentence: 1', '있지만', 'Adjective', '70'],
 ['Sentence: 1', '일', 'Noun', '70'],
 ['Sentence: 1', '을', 'Josa', '70'],
 ['Sentence: 1', '배우기에', 'Verb', '70'],
 ['Sentence: 1', '좋은', 'Adjective', '70'],
 ['Sentence: 1', '병원', 'Noun', '70'],
 ['Sentence: 2', '뚜렷한', 'Adjective', '70'],
 ['Sentence: 2', '목표', 'Noun', '70'],
 ['Sentence: 2', '의식', 'Noun', '70'],
 ['Sentence: 2', '이', 'Josa', '70'],
 ['Sentence: 2', '있다면', 'Adjective', '70'],
 ['Sentence: 2', '좋은', 'Adjective', '70'],
 ['Sentence: 2', '경험', 'Noun', '70'],
 ['Sentence: 2', '과', 'Josa', '70'],
 ['Sentence: 2', '경력', 'Noun', '70'],
 ['Sentence: 2', '이', 'Josa', '70'],
 ['Sentence: 2', '될', 'Verb', '70'],
 ['Sentence: 2', '수', 

In [275]:
dict_pos=[]
dict_neg=[]
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/pos_pol_word.txt", 'rt', encoding='UTF8'):
    dict_pos.append(line.split('\n')[0])
    
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/neg_pol_word.txt", 'rt', encoding='UTF8'):
    dict_neg.append(line.split('\n')[0])

In [276]:
dict_pos

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 4863',
 '극성 : 긍정',
 '',
 '(-;',
 '(^^)',
 '(^-^)',
 '(^^*',
 '(^_^)',
 '(^o^)',
 '*^^*',
 '/^o^\\',
 ':(',
 ":'-(",
 ':-(',
 ':-)',
 ':|',
 ';',
 ';)',
 ';-)',
 '^///^',
 '^^',
 '^^//',
 '<3',
 '=)',
 'XD',
 'ㄱㅅ',
 'ㄱㅇㄷ',
 '가격이 싸다',
 '가까이 사귀어',
 '가까이하다',
 '가다듬어',
 '가다듬어 수습하는',
 '가다듬어 수습하다',
 '가다듬어 정하다',
 '가당히',
 '가뜬하게',
 '가라앉혀 바로잡다',
 '가르치며',
 '가르침',
 '가벼운 마음으로',
 '가볍게 웃

In [277]:
dict_neg

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 9826',
 '극성 : 부정',
 '',
 '가난',
 '가난뱅이',
 '가난살이',
 '가난살이하다',
 '가난설음',
 '가난에',
 '가난에 쪼들려서',
 '가난하게',
 '가난하고',
 '가난하고 어렵다',
 '가난하고 천한',
 '가난하다',
 '가난하여',
 '가난한',
 '가난한 데다가',
 '가난한 사람을',
 '가난히',
 '가년스럽다',
 '가녕스럽다',
 '가누지 못하고',
 '가누지 못하다',
 '가누지 못할',
 '가누지 못할 정도로',
 '가누지 못함',
 '가눌 수 없게',
 '가눌 수 없을',
 '가눌 수 없이',
 '가당찮다',
 '가당찮이',
 '가두거나 해치거나',
 '가망이 없게',
 '가슴 아파하다',
 '가슴 아파함',

In [278]:
data = pd.DataFrame(text1_sent) # data를 pandas로 변환

In [279]:
data.columns = ['Sentence #', 'Word', 'POS', 'score']

In [280]:
data.head(10)

,Sentence #,Word,POS,score
0,Sentence: 1,태백,Noun,70
1,Sentence: 1,에,Josa,70
2,Sentence: 1,있다는,Adjective,70
3,Sentence: 1,위치,Noun,70
4,Sentence: 1,적,Suffix,70
5,Sentence: 1,인,Josa,70
6,Sentence: 1,단점,Noun,70
7,Sentence: 1,은,Josa,70
8,Sentence: 1,있지만,Adjective,70
9,Sentence: 1,일,Noun,70


In [281]:
data.tail(10)

,Sentence #,Word,POS,score
5482,Sentence: 300,충청북도,Noun,64
5483,Sentence: 300,진천군,Noun,64
5484,Sentence: 300,광혜원면,Noun,64
5485,Sentence: 300,에,Josa,64
5486,Sentence: 300,위치,Noun,64
5487,Sentence: 300,한,Josa,64
5488,Sentence: 300,기상청,Noun,64
5489,Sentence: 300,산하,Verb,64
5490,Sentence: 300,기관,Noun,64
5491,Sentence: 300,.,Punctuation,64


In [282]:
data.isnull().sum() # 비어있는 값 확인 결과: 없음

Sentence #    0
Word          0
POS           0
score         0
dtype: int64

In [283]:
words = list(set(data["Word"].values))
n_words = len(words) # 고유한 단어들이 1784개(문장부호 포함)
n_words

1785

In [284]:
# 데이터에서 문장을 원하는 형태로 포매팅하여 추출하는 클래스
# Word, POS -> data를 여러 함수로 씌워서 봄. 
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        # for문 사용법 보기. lambda: 일종의 함수(적용하는), data에 다음과 같은 기능 적용
        agg_func = lambda s: [(w, p) for w, p in zip(s["Word"].values.tolist(),
                                                     s["POS"].values.tolist())]
        # sentence 단위로 grouped
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        # 그룹화된 데이터를 다시 sentences로 저장
        self.sentences = [s for s in self.grouped]
        print(self.grouped)
        self.sentences
    
    # get_next: 다음 문장 처리
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [285]:
# 전체 문장 추출
getter = SentenceGetter(data)

Sentence #
Sentence: 1      [(태백, Noun), (에, Josa), (있다는, Adjective), (위치,...
Sentence: 10     [(재외공관, Noun), (대표, Noun), (부, Noun), (에서, Jos...
Sentence: 100    [(정부, Noun), (부처, Noun), (근무, Noun), (의, Josa)...
Sentence: 101    [(쓸데없는거에만, Adjective), (체계, Noun), (를, Josa), ...
Sentence: 102    [(조금, Noun), (특별하면서, Adjective), (보수, Noun), (...
                                       ...                        
Sentence: 95     [(여의도, Noun), (최저임금, Noun), (..., Punctuation)...
Sentence: 96     [(MICE, Alpha), (산업, Noun), (이, Josa), (어떻게, A...
Sentence: 97     [(외국, Noun), (계, Suffix), (처럼, Josa), (보이, Nou...
Sentence: 98     [(첫, Noun), (직장, Noun), (으로도, Josa), (괜찮고, Adj...
Sentence: 99     [(좋아요, Adjective), (야근, Noun), (강요, Noun), (없고...
Length: 300, dtype: object


In [286]:
# 문장 하나
sent = getter.get_next()

In [287]:
# 추출한 문장 확인(적용 결과)
print(sent)

[('태백', 'Noun'), ('에', 'Josa'), ('있다는', 'Adjective'), ('위치', 'Noun'), ('적', 'Suffix'), ('인', 'Josa'), ('단점', 'Noun'), ('은', 'Josa'), ('있지만', 'Adjective'), ('일', 'Noun'), ('을', 'Josa'), ('배우기에', 'Verb'), ('좋은', 'Adjective'), ('병원', 'Noun')]


In [292]:
# 전체 문장
sentences = getter.sentences

In [294]:
print(len(sentences)) # 개수 확인

# 문장 하나 확인
print(sentences[58])
print(sentences[:2]) # list의 list 형태로 여러 개 표시. 

# 각각 word, pos, tag, pare로 만들어서 결과 확인

300
[('정', 'Noun'), ('출연', 'Noun'), ('공공기관', 'Noun'), ('이라서', 'Josa'), ('연차', 'Noun'), ('자유', 'Noun'), ('롭고', 'Josa'), ('또한', 'Noun'), ('연', 'Modifier'), ('구직', 'Noun'), ('이', 'Josa'), ('대부분', 'Noun'), ('이라', 'Josa'), ('자유로운', 'Adjective'), ('분위기', 'Noun'), ('같습니다', 'Adjective')]
[[('태백', 'Noun'), ('에', 'Josa'), ('있다는', 'Adjective'), ('위치', 'Noun'), ('적', 'Suffix'), ('인', 'Josa'), ('단점', 'Noun'), ('은', 'Josa'), ('있지만', 'Adjective'), ('일', 'Noun'), ('을', 'Josa'), ('배우기에', 'Verb'), ('좋은', 'Adjective'), ('병원', 'Noun')], [('재외공관', 'Noun'), ('대표', 'Noun'), ('부', 'Noun'), ('에서', 'Josa'), ('계약', 'Noun'), ('직', 'Noun'), ('으로', 'Josa'), ('근무', 'Noun'), ('.', 'Punctuation'), ('태극마크', 'Noun'), ('라는', 'Josa'), ('자부심', 'Noun'), ('으로', 'Josa'), ('일', 'Noun'), ('하는', 'Verb'), ('조직', 'Noun')]]


In [295]:
# organization 정하여서 특징 정하기
#dict_org = ["특정한 키워드!", "정","하","기"] 
# 캐릭터 개수
# '개월' 관련 키워드
# dict_time = ["월", "달", "년", "일", "분", "시간", "초", "시", "날짜", "날", "오늘", "내일", "모레", "어제", "현재", "과거", "미래"]


# 여러가지 시도해보기!

# 단어를 features화
# "feat의 이름" : "value"
# 하나의 단어에서 다양한 feature(단어, POS tag, lables 등) 확인을 위해 문장 전체를 넣어줌. 
def word2features(sent, i):
    word = sent[i][0]   # index0의 단어를 가져옴
    postag = sent[i][1] # index1의 단어를 가져옴

# postag 유지
# postag2 = kkma.pos(word)
# 
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(), # 'feature의 이름' : 함수
        'word[-3:]': word[-3:], # 마지막 세 음절
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        #'dict_time': word.lower() in dict_time # -> true or false로
# character 개수 
# 'word.len' : len(word)
# 'word.len, num' : 
    }
    
    # 직전의 단어에 대해서도 똑같이 적용
    if i > 0:
        word1 = sent[i-1][0] # i-1: 직전의 단어
        postag1 = sent[i-1][1]
        # update: 기존에서 추가! (하여 새로운 dictionary를 만드는 결과)
        features.update({
            '-1:word.lower()': word1.lower(), # -1: 전 단어
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True # BOS: Beginning Of Sentence

    # 다음 단어에 대해서도 똑같이 처리
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True  # EOS: end of sentence

    return features

# sentence 전체에 적용하기 위한 함수
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))] # 0~i(index의 수)까지 적용 -> sentence에 적용
# == features = [word2features(sent, i) for i in range(len(sent))]
# return features

def sent2labels(sent): # sent를 읽어서 for문을 돌림. label만 갖고 옴
    return [label for token, postag, label in sent]
# 지금은 3개가 tuple로 있음.

def sent2tokens(sent):
    return [token for token, postag, label in sent]